# 6.1.2 Surface PDEs
## Surface Poisson equation

### Closed surface without boundary

Given a sphere we want to solve the surface Poisson equation 
$$\int_S(\nabla_{\Gamma}u\cdot\nabla_{\Gamma}v + uv)\,ds = \int_S fv\,ds$$


In [ ]:
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve import *
from ngsolve.webgui import Draw
ngsglobals.msg_level = 1

# Generate mesh and plot it
center = Pnt(0,0,0)
R = 1.0
geo = CSGeometry()
geo.Add(Sphere(center, R))
maxh = 1.0
mesh = Mesh(geo.GenerateMesh(maxh=maxh, perfstepsend=MeshingStep.MESHSURFACE))
Draw(mesh)

Note that the surface mesh is not very smooth as it is only a piecewise linear approximation
of the sphere geometry. To get a better geometric approximation, we can curve the mesh,
here we use 3rd order polynomials, i.e. ```order_g = 3```.

In [ ]:
order_k = 3
order_g = order_k
mesh.Curve(order_g)
Draw(mesh)

We solve the problem as usual with a minor twist:
For the (bi-)linear form we have to take care that we have to define the according integrators on the boundary and that the <i>Trace</i> operator has to be used to obtain the tangential/surface derivative.

In [ ]:
fes = H1(mesh, order=order_k)
u, v = fes.TnT()
a = BilinearForm(fes)
a += (grad(u).Trace()*grad(v).Trace() + u.Trace()*v.Trace())*ds
a.Assemble()

force = sin(x)*sin(y)*sin(z)

f = LinearForm(fes)
f += force*v*ds
f.Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec
Draw(gfu, mesh, "u")

If we want to write the solution into a vtk file, we need to set the ```vb=BND``` in the ```.Do()``` call:

In [ ]:
vtkfile = VTKOutput(mesh, coefs=[gfu, force], names=["uh", "f"], filename="surface_poisson",subdivision=3)
vtkfile.Do(vb=BND)



### Homogeneous Dirichlet data

Given a half-sphere we want to solve the surface Poisson equation $$\int_S\nabla_{\Gamma}u\cdot\nabla_{\Gamma}v\,ds = \int_S fv\,ds,\qquad u=0 \text{ on } \Gamma_D$$

First we define the geometry and generate the corresponding mesh. Since
we use AddSurface to add a surface geometry, which is cut automatically with the base geometry. Here, no additional flag needs to be set in the GenerateMesh method.
See the [ this i-tutorial](https://docu.ngsolve.org/nightly/i-tutorials/unit-6.1.1-surfacemeshes/surface_meshes.html) for more information regarding surface meshing.

In [ ]:
geo          = CSGeometry()
sphere       = Sphere(Pnt(0,0,0), 1)
bot          = Plane(Pnt(0,0,0), Vec(0,0,-1))
finitesphere = sphere * bot

geo.AddSurface(sphere, finitesphere.bc("surface"))
geo.NameEdge(sphere,bot, "bottom")

mesh = Mesh(geo.GenerateMesh(maxh=0.3))
mesh.Curve(2)
Draw(mesh)

Therefore, we define the usual $H^1$ finite element space and use the <i>dirichlet_bbnd</i> flag to indicate the BBoundary on which the Dirichlet conditions are prescribed. The test- and trial-functions are given as usual.

In [ ]:
fes = H1(mesh, order=2, dirichlet_bbnd="bottom")
u, v = fes.TnT()
print(fes.FreeDofs())

For the (bi-)linear form we have to take care that we have to define the according integrators on the boundary and that the <i>Trace</i> operator has to be used to obtain the tangential/surface derivative

In [ ]:
a = BilinearForm(fes, symmetric=True)
a += grad(u).Trace()*grad(v).Trace()*ds
a.Assemble()

force = sin(x)*y*exp(z)

f = LinearForm(fes)
f += force*v*ds
f.Assemble()

Solving is done as usual

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec
Draw(gfu, mesh, "u")

### Inhomogeneous Dirichlet data


To solve the same problem with non-homogenous Dirichlet data, $u=u_D$ on $\Gamma_D$ the same technique as in the volume case is used, where we have to set a function on the BBoundary instead on the boundary

In [ ]:
gfu.Set(x, definedon=mesh.BBoundaries("bottom"))
r = f.vec.CreateVector()
r.data = f.vec - a.mat*gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs())*r
Draw(gfu)

## Finite element spaces for surfaces


We differ between two types of finite element spaces for surfaces. The first class consists of spaces, where the restriction of a 3D element to the surface leads to a valid 2D element of the same type: H1, HCurl, HCurlCurl, NumberSpace.

These spaces can directly be used, one has to take care using the Trace operator. Otherwise an exception is thrown during assembling. 

In [ ]:
a += grad(u)*grad(v)*ds
a.Assemble()

The NumberSpace is an exception as it represents only a number, where no Trace operator has to be used.

The second class is given by

| Space     |  Surface Space |
|:----------|:---------------|
| L2 	       | SurfaceL2
| HDiv         | HDivSurface
| HDivDiv 	   | HDivDivSurface
| FacetFESpace | FacetSurface

Here, a 2D reference element is mapped directly onto the surface. To be consistent, also here the Trace operator has to be used.